In [ ]:

%%capture
!pip install -U librosa
!pip install pydub

In [ ]:
import pandas as pd

import librosa
import numpy as np
import pywt

from pydub import AudioSegment

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# root_folder = '/content/gdrive/MyDrive/University/master/term1/Machine-Learning/ML-Project/ML_prj_2023'
# res_folder = '/content/gdrive/MyDrive/University/master/term1/Machine-Learning/ML-Project/Data/mfcc/'
# res_folder_full = '/content/gdrive/MyDrive/University/master/term1/Machine-Learning/ML-Project/Data/ML_prj_2023_wav_full'

In [ ]:
root_folder = "/content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023"
res_folder = "/content/gdrive/MyDrive/Tehran University/ML/ML-Project/Data/mfcc"

# Extract File_names

In [ ]:
import glob
import os
 
# Path of the directory
dir_path = root_folder
 
# Get the list of all the files in directory tree at given path
listOfFiles = list()
for (dirpath, dirnames, filenames) in os.walk(dir_path):
    for file in filenames:
        if '.csv' not in file:
            listOfFiles.append(os.path.join(dirpath, file))
            print(int(os.path.join(dirpath, file).split(os.sep)[-3].split("_")[1]),os.path.join(dirpath, file))
        else:
            print(os.path.join(dirpath, file))

# Print the list of all files
print('List of all files = ', len(listOfFiles))
listOfFiles[1:10]

/content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/mlproject.csv
2 /content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/ney-mahoor-ebrahimi .mp3
2 /content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/Mohammad_Shojaei_nei_Mahoor.mp3
2 /content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/Arash_Samimi_nei_Mahoor.mp3
2 /content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/19 sarebaang mahoor.mp3
2 /content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/Amjadian.mp3
2 /content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/rasoul_akbari_arragh.mp3
2 /content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/15084420252377mp3-72-8_new.mp3
2 /content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/Y2Mate.is - تکنوازی نی امین رحیمی ،بداهه در دستگاه ماهور-_-ZZ7LGL6hI-160k-1655953757264.mp3
2 /content/gdrive/MyDriv

['/content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/Mohammad_Shojaei_nei_Mahoor.mp3',
 '/content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/Arash_Samimi_nei_Mahoor.mp3',
 '/content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/19 sarebaang mahoor.mp3',
 '/content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/Amjadian.mp3',
 '/content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/rasoul_akbari_arragh.mp3',
 '/content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/15084420252377mp3-72-8_new.mp3',
 '/content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/Y2Mate.is - تکنوازی نی امین رحیمی ،بداهه در دستگاه ماهور-_-ZZ7LGL6hI-160k-1655953757264.mp3',
 '/content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_2/I_4/Hassan_Kassaie-Mahour_new_new_new-(WWW (mp3cut.net).mp3',
 '/content/gdrive/MyDrive/Tehran University/ML/ML-Project/M

# Load audio file

In [ ]:
def load_audio(audio_path):
  y, sr = librosa.load(audio_path)
  return y, sr

# Audio Data cleaning

In [ ]:
# Removing silence
# Normalization
# Noise reduction
# Resampling

In [ ]:
def wavelet_denoising(y, wavelet='db8', level=1):
  
    # Perform wavelet decomposition
    coeffs = pywt.wavedec(y, wavelet, level=level)
    
    # Identify and remove noisy components
    sigma = np.median(np.abs(coeffs[-level])) / 0.6745
    threshold = sigma * np.sqrt(2 * np.log(len(y)))
    coeffs[1:] = [pywt.threshold(i, value=threshold, mode='hard') for i in coeffs[1:]]

    # Reconstruct the signal using inverse wavelet transform
    denoised_y = pywt.waverec(coeffs, wavelet)

    # print("denoised")

    return denoised_y

In [ ]:
def clean_audio(y, sr, is_noise_reduction=False):
  
  # Resample audio
  # y = librosa.resample(y, orig_sr=sr, target_sr=44100)
  # sr = 44100

  # # Remove silence
  # y, index = librosa.effects.trim(y, top_db=30)

  # Normalize audio
  y = librosa.util.normalize(y)

  if is_noise_reduction:
    y = wavelet_denoising(y)

  return y, sr

# Audio conversion

In [ ]:
def mp3_to_wav(src_path, out_path):
  sound = AudioSegment.from_mp3(src_path)
  sound.export(out_path, format="wav")

In [ ]:
def audio_to_wav(src_path, out_path):
  sound = AudioSegment.from_file(src_path)
  sound.export(out_path, format="wav")

# Feature Extraction

In [ ]:
def feature_extractor(y, sr , mehod_func, is_clean_audio=False, is_noise_reduction=False):

  if is_clean_audio:
    y, sr = clean_audio(y,sr,is_noise_reduction)

  extracted_feature = mehod_func(y=y, sr=sr)
  extracted_feature = np.transpose(extracted_feature)

  return extracted_feature

In [ ]:
def feature_extractor_all(listOfFiles, method_func, is_clean_audio=False, is_noise_reduction=False, split_seconds=None):
    extracted_feature_files = []
    data = []
    labels = []
    for index,file_path in enumerate(listOfFiles):
        sound = AudioSegment.from_file(file_path)
        soundLength = len(sound)
        numberOfParts = soundLength // split_seconds
        dastgah = int(file_path.split(os.sep)[-3].split("_")[1])
        if numberOfParts > 0:
            loss = soundLength - numberOfParts * split_seconds
            skipEnd = loss // 2
            skipStart = loss - skipEnd
            start = skipStart + 1
        else:
            print(file_path, soundLength)
            continue
        # print(index, numberOfParts, dastgah, file_path)
        for i in range(numberOfParts):
            last_part = min(start+split_seconds+1,soundLength)
            part = sound[start: last_part]
            part.export('test.wav' , format="wav")
            part = AudioSegment.from_file('test.wav')
            # raw_audio = part.get_array_of_samples()
            y, sr = librosa.load('test.wav', sr=22050)
            extracted_audio_feature  = feature_extractor(y , sr, method_func, is_clean_audio, is_noise_reduction)
            # audio_duration = librosa.get_duration(y=y, sr=sr)

            if extracted_audio_feature.shape[0] != 862:
              # print(file_path)
              print(extracted_audio_feature.shape)
              extracted_audio_feature = np.vstack([extracted_audio_feature, np.zeros((n_chroma))])

              print("Shape afterward")
              print(extracted_audio_feature.shape)

              # print(extracted_audio_feature)

            # print(extracted_audio_feature.shape)
            data.append(extracted_audio_feature)
            labels.append(dastgah)
            start = start+split_seconds+1



    return data, labels
      

# Mel-frequency cepstral coefficients (MFCCs)

## Sample_code

In [ ]:
# n_mfcc = 40
# feature_extraction_method = lambda y, sr: librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc,n_fft = 2048, hop_length = 512)

In [ ]:
n_chroma = 12
feature_extraction_method = lambda y, sr: librosa.feature.chroma_stft(y=y, sr=sr, n_chroma=n_chroma ,n_fft = 2048, hop_length = 512)

In [ ]:
data, labels = feature_extractor_all(listOfFiles, feature_extraction_method, True, False, 20000)

(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)
/content/gdrive/MyDrive/Tehran University/ML/ML-Project/ML_prj_2023/D_4/I_1/ravandi.mp3 16327
(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)
(861, 12)
Shape afterward
(862, 12)


In [ ]:
np.unique(labels)

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
print(np.shape(data[0]), np.shape(data[1]), np.shape(data[2]), np.shape(data[3]), np.shape(data[4]))

(862, 12) (862, 12) (862, 12) (862, 12) (862, 12)


In [ ]:
print(data[0])

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.09784973 0.00842196 0.01446803 ... 0.01580771 0.543029   1.        ]
 [0.91443884 0.09261242 0.0239588  ... 0.01873619 0.33117348 1.        ]
 [1.         0.1522657  0.01815607 ... 0.00525323 0.02328316 0.32185027]]


In [ ]:
print(labels[0:5])

[2, 2, 2, 2, 2]


## Saving in file

In [ ]:
file_name  = res_folder + "audio_chroma_features_12_all_20Sec_cleaned.pkl"
file_name_labels  = res_folder + "audio_chroma_features_12_all_20Sec_labels_cleaned.pkl"

In [ ]:
import pickle

with open(file_name, 'wb') as file:
    pickle.dump(data, file)

with open(file_name_labels, 'wb') as file:
    pickle.dump(labels, file)

In [ ]:
with open(file_name, 'rb') as file:
    loaded_data = pickle.load(file)

In [ ]:
drive.flush_and_unmount()

In [ ]:
loaded_data.shape

# Chroma feature

## Sample code

In [ ]:
chroma = librosa.feature.chroma_stft(y=y, sr=sr)
chroma = np.transpose(chroma)

In [ ]:
chroma.shape

In [ ]:
chroma_audio_files_features = feature_extractor_all(file_names, librosa.feature.chroma_stft)

In [ ]:
import pickle

with open('audio_chroma_features.pkl', 'wb') as file:
    pickle.dump(chroma_audio_files_features, file)

In [ ]:
!cp /content/audio_chroma_features.pkl /content/gdrive/MyDrive/Tehran\ University/ML/ML-Project/Data

In [ ]:
drive.flush_and_unmount()